# === Kapampangan-to-English NLLB-200 Training Pipeline (with <kap> tag) ===

In [87]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
import torch
import evaluate
import numpy as np

# === 1. Config ===

In [88]:
CSV_PATH = "data/kapampangan_english.csv"
MODEL_NAME = "facebook/nllb-200-distilled-600M"
MODEL_DIR = "./kapampangan_mt_nllb"

SPECIAL_SRC_TOKEN = "<kap>"   # Custom Kapampangan language marker
TGT_LANG = "eng_Latn"         # English (Latin script)

# === 2. Load CSV ===

In [89]:
df = pd.read_csv(CSV_PATH)
df = df.rename(columns={"kapampangan": "src_text", "english": "tgt_text"})
df = df.dropna(subset=["src_text", "tgt_text"])

# === 3. Convert to HF Dataset ===

In [90]:
dataset = Dataset.from_pandas(df[["src_text", "tgt_text"]])
dataset = dataset.train_test_split(test_size=0.2, seed=42)

# === 4. Load Tokenizer & Model ===

In [91]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)  # No src_lang yet
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Add <kap> as a special token
tokenizer.add_special_tokens({'additional_special_tokens': [SPECIAL_SRC_TOKEN]})
model.resize_token_embeddings(len(tokenizer))

# Force English output
model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(TGT_LANG)


# === 5. Preprocess ===

In [92]:
def preprocess(examples):
    # Prepend <kap> to source text
    src_texts = [f"{SPECIAL_SRC_TOKEN} {text}" for text in examples["src_text"]]

    # Tokenize source
    model_inputs = tokenizer(
        src_texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Tokenize target
    labels = tokenizer(
        examples["tgt_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )["input_ids"]

    # Replace pad token IDs with -100
    labels = [[(t if t != tokenizer.pad_token_id else -100) for t in label] for label in labels]
    model_inputs["labels"] = labels

    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/232 [00:00<?, ? examples/s]

# === 6. Training Args ===

In [95]:
metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)

    # Replace -100 with pad_token_id so metric can handle it
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    accuracy = metric_acc.compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy["accuracy"]}

training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_DIR,
    learning_rate=1e-4,
    evaluation_strategy="steps",   # or "epoch"
    eval_steps=500,                 # eval every 500 steps
    logging_strategy="steps",       # log every X steps
    logging_steps=100,              # log every 100 steps
    save_strategy="steps",
    save_steps=500,
    load_best_model_at_end=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),                   # True if GPU supports it
)

TypeError: Seq2SeqTrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

# === 7. Trainer ===

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


# === 8. Train ===

In [85]:
trainer.train()

C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


KeyboardInterrupt: 

# === 9. Save ===

In [ ]:
trainer.save_model(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)
print(f"✅ Model saved to: {MODEL_DIR}")

# === 10. Translation Function ===

In [ ]:
def batch_translate(texts, batch_size=8):
    results = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    for i in range(0, len(texts), batch_size):
        src_texts = [f"{SPECIAL_SRC_TOKEN} {t}" for t in texts[i:i+batch_size]]
        inputs = tokenizer(src_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        with torch.no_grad():
            outputs = model.generate(**inputs)
        results.extend(tokenizer.batch_decode(outputs, skip_special_tokens=True))
    return results

# === 11. Evaluate BLEU ===

In [ ]:
print("\n--- Evaluating BLEU Score ---")
bleu = evaluate.load("bleu")

test_df = dataset["test"].to_pandas()
preds = batch_translate(test_df["src_text"].tolist())
refs = [[x] for x in test_df["tgt_text"].tolist()]

bleu_score = bleu.compute(predictions=preds, references=refs)
print(" BLEU Score:", bleu_score)

# === 12. Manual Test ===

In [ ]:
print("\n--- Manual Test ---")
sample_texts = [
    "Ali ku balu",
    "Anya ka?",
    "Masanting ya ing panaun ngeni",
    "E ku makanyan",
]

for i, kap_text in enumerate(sample_texts):
    translated = batch_translate([kap_text])[0]
    print(f"[{i+1}] Kapampangan: {kap_text}")
    print(f"    ➤ English: {translated}")